*2021/01/12 Tue*

<hr>

# 9-1. 코드를 찍어내는 틀 - C++ 템플릿(template)

## C++ 템플릿(template)

컴파일러는 템플릿 인자를 보고 치환된 코드를 생성한다. 이렇게 클래스 템플릿에 인자를 전달해서 실제 코드를 생성하는 것을 <u>클래스 템플릿 인스턴스화</u>(class template instantiation)라 한다.

템플릿이 인스턴스화되지 않고 덩그러니 있다면 컴파일 시에 아무런 코드로 변환되지 않음.

In [ ]:
#include <iostream>
#include <string>

template <typename T>  // <class T>로 써도 동일. 그러나 원래의 것을 권장.
class Vector {
    T *data;
    int capacity;
    int length;
    
public:
    // 생성자
    Vector(int n = 1) : data(new T[n]), capacity(n), length(0) {}
    
    // 맨 뒤에 새로운 원소를 추가한다.
    void push_back(T s) {
        if (capacity <= length) {
            T *temp = new T[capacity * 2];
            for (int i = 0; i < length; i++) {
                temp[i] = data[i];
            }
            delete[] data;
            data = temp;
            capacity *= 2;
        }
        
        data[length] = s;
        length++;
    }
    
    // 임의의 위치의 원소에 접근한다.
    T operator[](int i) { return data[i]; }
    
    // x 번째 위치한 원소를 제거한다.
    void remove(int x) {
        for (int i = x + 1; i < length; i++) {
            data[i - 1] = data[i];
        }
        length--;
    }
    
    // 현재 벡터의 크기를 구한다.
    int size() { return length; }
    
    ~Vector() {
        if (data) {
            delete[] data;
        }
    }
};

int main() {
    // int를 보관하는 벡터를 만든다.
    Vector<int> int_vec;
    int_vec.push_back(3);
    int_vec.push_back(2);
    
    std::cout << "-------- int vector --------" << std::endl;
    std::cout << "첫 번째 원소 : " << int_vec[0] << std::endl;
    std::cout << "두 번째 원소 : " << int_vec[1] << std::endl;
    
    Vector<std::string> str_vec;
    str_vec.push_back("hello");
    str_vec.push_back("world");
    std::cout << "-------- std::string vector --------" << std::endl;
    std::cout << "첫 번째 원소 : " << str_vec[0] << std::endl;
    std::cout << "두 번째 원소 : " << str_vec[1] << std::endl;
}

## 템플릿 특수화(template specialization)

위 코드에서 템플릿 인자에 ```bool```을 전달할 수도 있을 것. 그런데 이것은 1비트 하나로도 충분히 저장할 수 있는데 반해, C++에서 기본으로 처리하는 단위는 1바이트이므로, 엄청난 메모리 낭비가 아닐 수 없음. 따라서 우리는 ```Vector<bool>```에 대해서는 특별히 따로 처리해 주어야 할 것이다. 왜? ```unsigned int``` 배열이면 32개의 ```bool```을 처리할 수 있으니까.

오버로딩!
```
template <typename T>
class Vector { ... };

template <>
class Vector<bool> { ... };
```

```N``` 번째 ```bool``` 데이터는 ```N / 32``` 번째 ```int```에 들어가 있고, 그 안에서 정확히 ```N % 32``` 번째 비트가 되도록 구현하면 된다.

그리고 실제로, C++ 표준 라이브러리의 ```vector``` 역시 ```bool```만 따로 특수화시켜서 처리하고 있음.

In [ ]:
template <typename A, typename B, typename C>
class test {};

// A가 int고, C가 double일 때 따로 처리하고 싶다면,
template <typename B>
class test<int, B, double> {};

// B조차 특수화하고 싶다면,
template <>
class test<int, int, double> {};

// 즉, bool 벡터를 특수화하고 싶다면,
template <>
class Vector<bool> {
    // ...
};

## 함수 템플릿(function template)

In [ ]:
template <typename T>
T max(T& a, T& b) { return a > b ? a : b; }

int a = 1, b = 2;
max(a, b);  // max<int>(a, b); : C++ 컴파일러가 자동으로 인스턴스화 해줌.

연산자 오버로딩이나 어떤 멤버 함수를 이용한 동작을 내부에서 하는데, 템플릿 인자로 넘긴 타입에서 그런 것이 정의되어 있지 않다면 컴파일 타임에 오류 발생시킴. 왜냐하면 컴파일 시에 모든 템플릿을 실제 코드로 변환하여 실행하기 때문.

또한 컴파일 시에 모든 템플릿이 인스턴스화된다는 사실을 갖고 여러 가지 흥미로운 코드를 짤 수 있는데, 이러한 방식을 템플릿 메타프로그래밍(template metaprogramming)이라고 함.

## 함수 객체(function object - functor)의 도입

In [ ]:
// 중요한 것 : comp는 함수가 아니라 객체이고, Comp 클래스에서 () 연산자를 오버로딩한 버전이다.

template <typename Cont, typename Comp>
void bubble_sort(Cont& cont, Comp& comp) {
    for (int i = 0; i < cont.size(); i++) {
        for (int j = i + 1; j < cont.size(); j++) {
            if (!comp(cont[i], cont[j])) {
                cont.swap(i, j);
            }
        }
    }
}

struct Comp1 {
    bool operator()(int a, int b) { return a > b; }
};
struct Comp2 {
    bool operator()(int a, int b) { return a < b; }
};

이렇게 함수는 아니지만 함수인 척 하는 객체를 <u>함수 객체(function object; `Functor`)</u>라고 한다. 이것 덕분에, `bubble_sort` 함수 내에서 두 객체 간의 비교를 사용자가 원하는 대로 할 수 있게 됨.

그리고 C에서 함수 포인터로 함수를 받아 처리하는 것보다 이게 나은 방법임. 전자는 컴파일러가 최적화를 수행할 수 없다. 그런데, `Functor`를 넘기게 되면 컴파일러가 `operator()` 자체를 인라인화시켜서 매우 빠르게 작업을 수행할 수 있음.

> 실제로 C의 qsort와 C++의 표준 sort 함수를 비교하면 C++ 버전이 훨씬 빠르다.

## 타입이 아닌 템플릿 인자(non-type template arguments)

템플릿 인자로 타입만 받을 수 있는 것은 아니다.

In [ ]:
template <typename T, int num>  //
T add_num(T t) {
    return t + num;
}

int main() {
    int x = 3;
    add_num<int, 5>(x);  //
}

위 코드에서 템플릿 인자 `<>`를 지정하지 않으면 컴파일러가 `num`에 무엇이 들어가는지 모르니 컴파일 오류를 띄움.

그리고 템플릿 인자로 전달할 수 있는 타입들이 제한적.
* `bool`, `char`, `int`, `long` 등 정수 타입들.(`float`, `double`은 제외)
* 포인터 타입
* `enum` 타입
* `std::nullptr_t` (널 포인터)

In [ ]:
// C에서의 배열(int arr[5] = {1, 2, 3, 4, 5};)처럼 {}을 통해 배열을 정의할 수 있음.
#include <array>

std::array<int, 5> arr = {1, 2, 3, 4, 5};  // int 타입, 크기 5를 템플릿 인자로 명시한 뒤 초기화.
// {} : uniform initialization이라 하여 C++ 11에서 추가된 개념.

// 이 arr은 런타임에서 동적으로 크기가 할당되는 것이 아님. 마치 배열처럼 int 5개를 갖는 메모리를 가지고 스택에 할당.

중요한 점 : 위에서의 `arr` 배열을 함수에 전달하기 위해서는 그냥 `std::array`를 받는 함수를 만들면 안 된다. `std::array<int, 5>` 자체가 하나의 타입이기 때문.

In [ ]:
void print_array(const std::array<int, 5>& arr) {
    for (int i = 0; i < arr.size(); i++)
        std::cout << arr[i] << " ";
    std::cout << std::endl;
}

// 그렇지만 이러면 array 크기별로 함수를 만들어 주어야 하니까, 이렇게 하지 말고 그냥 여기서도 템플릿 쓰면 됨.

template <typename T>
void print_array(const T& arr) {
    for (int i = 0; i < arr.size(); i++)
        std::cout << arr[i] << " ";
    std::cout << std::endl;
}

## 디폴트 템플릿 인자

함수에 디폴트 인자 지정할 수 있는 것처럼 템플릿도 디폴트 인자를 지정할 수 있다!

In [ ]:
#include <iostream>
#include <string>

template <typename T>
struct Compare {
    bool operator()(const T& a, const T& b) const { return a < b; }
};

template <typename T, typename Comp = Compare<T>>
T Min(T a, T b) {
    Comp comp;
    if (comp(a, b))
        return a;
    return b;
}

int main() {
    int a = 3, b = 5;
    std::cout << "Min " << a << " , " << b << " :: " << Min(a, b) << std::endl;

    std::string s1 = "abc", s2 = "def";
    std::cout << "Min " << s1 << " , " << s2 << " :: " << Min(s1, s2)
            << std::endl;
}

<hr>

# 9-2. 가변 길이 템플릿(variadic template)

## 가변 길이 템플릿

`typename` 뒤에 `...`으로 오는 것을 <u>템플릿 파라미터 팩(parameter pack)</u>이라고 함. 0개 이상의 템플릿 인자들을 나타낸다.

In [2]:
template <typename T>
void print(T arg) {
    std::cout << arg << std::endl;
}
template <typename T, typename... Types>  // ...로 오는 것을 템플릿 파라미터 팩이라 하고,
void print(T arg, Types... args) {  // 마찬가지로 ...로 오는 것을 함수 파라미터 팩이라 한다.
    std::cout << arg << ", ";
    print(args...);
}

int main() {
    print(1, 3.1, "abc");
    print(1, 2, 3, 4, 5, 6, 7);
}

C++ 규칙상, 파라미터 팩이 없는 함수의 우선 순위가 높음. 그래서 위의 함수로 빠져 나가게 됨.

그리고 만약 두 함수의 순서를 바꾼다면, 컴파일 오류. 컴파일러는 자신의 앞에 정의되어 있는 함수들밖에 보지 못하기 때문이다. 그리고 결과적으로 `print()`를 호출하는, 없는 함수를 호출하는 코드로 바뀌므로 오류가 발생하는 것.

그래서 템플릿 함수를 작성할 때에는 그 순서에 유의해야 한다.

## 임의의 개수의 문자열을 합치는 함수

`concat = s1 + s2 + s3;`는 `concat = s1.operator+(s2).operator+(s3);`와 같다.

그런데, 메모리 할당/해제는 매우 느린 작업 중 하나이므로, 다음과 같은 식으로 코드를 작성할 수 있음.

In [ ]:
std::string concat;
concat.reserve(s1.size() + s2.size() + s3.size());  // 여기서 할당 1 번 수행
concat.append(s1);
concat.append(s2);
concat.append(s3);

이걸 `std::string concat = StrCat(s1, "abc", s2, s3);`와 같이 깔끔하게 작성할 수 없을까? -> 가변 길이 템플릿!

In [ ]:
#include <cstring>
#include <iostream>
#include <string>

size_t GetStringSize(const char* s) { return strlen(s); }

size_t GetStringSize(const std::string& s) { return s.size(); }
template <typename String, typename... Strings>
size_t GetStringSize(const String& s, Strings... strs) {
    return GetStringSize(s) + GetStringSize(strs...);
}

void AppendToString(std::string* concat_str) { return; }

template <typename String, typename... Strings>
void AppendToString(std::string* concat_str, const String& s, Strings... strs) {
    concat_str->append(s);
    AppendToString(concat_str, strs...);
}

template <typename String, typename... Strings>
std::string StrCat(const String& s, Strings... strs) {
    // 먼저 합쳐질 문자열의 총 길이를 구한다.
    size_t total_size = GetStringSize(s, strs...);

    // reserve 를 통해 미리 공간을 할당해 놓는다.
    std::string concat_str;
    concat_str.reserve(total_size);

    concat_str = s;
    AppendToString(&concat_str, strs...);

    return concat_str;
}

int main() {
    // std::string 과 const char* 을 혼합해서 사용 가능하다.
    std::cout << StrCat(std::string("this"), " ", "is", " ", std::string("a"),
            " ", std::string("sentence"));
}

## sizeof...

파라미터 팩에 `sizeof...`을 사용하면 전체 인자의 개수를 리턴한다.

In [ ]:
int sum_all() { return 0; }
template <typename... Ints>
int sum_all(int num, Ints... nums) {
    return num + sum_all(nums...);
}

template <typename... Ints>
double average(Ints... nums) {
    return static_cast<double>(sum_all(nums...)) / sizeof...(nums);  //
}

## Fold Expression

C++ 11에서 도입된 가변 길이 템플릿은 매우 편리하지만, 재귀 함수 형태로 구성해야 하므로 반드시 재귀 호출 종료를 위한 함수를 따로 만들어야 한다는 것이 단점. -> C++ 17에 새로 도입된 Fold 형식을 사용하면 이를 훨씬 간단하게 표현할 수 있다.

In [ ]:
#include <iostream>

template <typename... Ints>
int sum_all(Ints... nums) {
    return (... + nums);  // Fold 형식
}

int main() {
    // 1 + 4 + 2 + 3 + 10
    std::cout << sum_all(1, 4, 2, 3, 10) << std::endl;
}

이항 좌측 Fold 식 `(... + nums)`는 `((((1 + 4) + 2) + 3) + 10)`와 같이 컴파일러에서 해석된다.

|이름|Fold 방식|실제 전개 형태|
|:---:|---|---|
|`(E op ...)`|단항(Unary) 우측 Fold|$$(E_{1}\,op\,(...\,op\,(E_{N-1}\,op\,E_{N})))$$|
|`(... op E)`|단항 좌측 Fold|$$(((E_{1}\,op\,E_{2})\,op\,...)\,op\,E_{N})$$|
|`(E op ... op I)`|이항 우측 Fold|$$(E_{1}\,op\,(...\,op\,(E_{N-1}\,op\,(E_{N}\,op\,I))))$$|
|`(I op ... op E)`|이항 우측 Fold|$$((((I\,op\,E_{1})\,op\,E_{2})\,op\,...)\,op\,E_{N})$$|

_($I$는 초깃값을 의미하며, 파라미터 팩이 아님.)_

Fold 식을 쓸 때는 꼭 `()`로 감싸주어야 한다. 그렇지 않으면 컴파일 오류.

In [ ]:
// 이항 Fold 예시(이항 좌측 Fold)
#include <iostream>

template <typename Int, typename... Ints>
Int diff_from(Int start, Ints... nums) {
    return (start - ... - nums);
}

int main() {
    // 100 - 1 - 4 - 2 - 3 - 10
    std::cout << diff_from(100, 1, 4, 2, 3, 10) << std::endl;
}

* `,` 연산자

In [ ]:
#include <iostream>

class A {
public:
    void do_something(int x) const {
        std::cout << "Do something with " << x << std::endl;
    }
};

template <typename T, typename... Ints>
void do_many_things(const T& t, Ints... nums) {
    // 각각의 인자들에 대해 do_something 함수들을 호출한다.
    (t.do_something(nums), ...);
}
int main() {
    A a;
    do_many_things(a, 1, 3, 2, 4);
}

\- `(t.do_something(nums), ...)`의 실행 결과

```
t.do_something(1);
t.do_something(3);
t.do_something(2);
t.do_something(4);
```

사실상 모든 인자들에 대해서 각각 `t.do_something(arg)`를 실행한 것과 같다!

$\therefore$ <u>가변 길이 템플릿을 잘 활용한다면 작성해야 하는 코드의 양을 줄일 수 있을 것!</u>

<br>

\+ *코드 컨벤션 참고*

![코드 컨벤션 참고](figures/Screen%20Shot%202021-01-12%20at%205.35.43%20AM.png)